Every Object-Oriented language has at least one standard way of getting a
string representation from any object.

Vector Class Redux
In order to demonstrate the many methods used to generate object
representations, we’ll use a Vector2d class similar to the one we saw in
Chapter 1.

Example 11-1. Vector2d instances have several representations

Example 11-2. vector2d_v0.py: methods so far are all special methods

In [7]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{} ({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)
        
    def __bool__(self):
        return bool(abs(self))



Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/amber/Desktop/Projects/2022/learnPythonWithBook/KartalOl-Fluent-Python-2022/venv10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/amber/Desktop/Projects/2022/learnPythonWithBook/KartalOl-Fluent-Python-2022/venv10/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [1]:
v1 = Vector2s(3, 4)

NameError: name 'Vector2s' is not defined